In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

/home/ximo/GitHub/skforecast


In [2]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

regressor = CatBoostRegressor()

In [3]:
type(regressor).__name__

'CatBoostRegressor'

In [4]:
def set_cpu_gpu_device(regressor: object, device: str | None = 'cpu') -> str:
    """
    Set the device for the regressor to either 'cpu', 'gpu', 'cuda', or None.
    
    Parameters
    ----------
    regressor : object
        An estimator object from XGBoost, LightGBM, or CatBoost.
    device : str, default 'cpu'
        The device to set. Can be 'cpu', 'gpu', or None. If None, it will
        use the current device of the regressor.
        
    Returns
    -------
    
    original_device : str
        The original device of the regressor before setting it to the new device.
    """

    device_names = {
        'XGBRegressor': 'device',
        'LGBMRegressor': 'device',
        'CatBoostRegressor': 'task_type',
    }

    device_values = {
        'XGBRegressor': {'gpu': 'cuda', 'cpu': 'cpu', 'cuda': 'cuda'},
        'LGBMRegressor': {'gpu': 'gpu', 'cpu': 'cpu'},
        'CatBoostRegressor': {'gpu': 'GPU', 'cpu': 'CPU'},
    }

    if device not in ['gpu', 'cpu', 'cuda', None]:
        raise ValueError("`device` must be 'gpu', 'cpu', 'cuda', or None.")

    regressor_type = type(regressor).__name__
    if regressor_type not in device_names:
        return None
    
    original_device = regressor.get_params()[device_names[regressor_type]].lower()
    param = device_names[regressor_type]
    value = device_values[regressor_type][device]
    try:
        regressor.set_params(**{param: value})
        return original_device
    except Exception as e:
        return original_device

In [5]:
regressor = CatBoostRegressor(task_type='GPU')
print(regressor.get_params()['task_type'])
original_device = set_cpu_gpu_device(regressor, 'cpu')
print(regressor.get_params()['task_type'])
set_cpu_gpu_device(regressor, original_device)
print(regressor.get_params()['task_type'])

GPU
CPU
GPU


In [6]:
regressor = XGBRegressor(device='cuda')
print(regressor.get_params()['device'])
original_device = set_cpu_gpu_device(regressor, 'cpu')
print(regressor.get_params()['device'])
set_cpu_gpu_device(regressor, original_device)
print(regressor.get_params()['device'])

cuda
cpu
cuda


In [7]:
regressor = LGBMRegressor(device='gpu')
print(regressor.get_params()['device'])
set_cpu_gpu_device(regressor, 'cpu')
print(regressor.get_params()['device'])
set_cpu_gpu_device(regressor, 'gpu')
print(regressor.get_params()['device'])

gpu
cpu
gpu
